
Nirvana Systems把Conners RSI称作终极技术指标，并在它的网站上发布了关于如何生成和使用这一指标的文章。著名的回测框架 backtrader也在文档和示例中引用了这篇文章。TradingView也内置了这一指标。

为什么Conners RSI被称为终极技术指标，它有哪些优势？成功背后的原理是什么，又该如何实现这一指标呢？这篇文章就为你介绍。

Conners RSI是在标准RSI的基础上，混合了另外两个指标得到的。

第一个指标就是Streaks。它本质上是统计连续上涨或者下跌的周期数，并将该指标的RSI作为一个分数。我们知道，股票连涨周期数越长，则越可能反生反转（即下跌）；反之亦然。大家可以按照我们在《数据分析与Python实现》那几节课中，介绍的PDF/CDF的方法，来自行估计当某个标的连续上涨N天后，接下来继续上涨的概率有多大。

下图显示了每日收盘价，及由此计算的streaks指标：

![50%](https://i.stack.imgur.com/SIQUq.png)

第二个指标就是今天的涨跌幅，在过去一段时间内的的涨跌幅中的排名。它从另一个维度，描绘了当前市场的强弱。如果在过去20天内，只有3天的涨幅低于今天，那么今天的相对强弱就是15%，次日反转可能性大；如果17天的涨幅低于今天，那么今天的相对强弱就是85%，次日下跌的可能性变大。

这两个指标加上经典RSI，就合成了Conners RSI:

$$
    CRSI(3, 2, 100) = [RSI(3) + RSI(Streak, 2) + PercentRank(100)] / 3
$$

与单一的RSI只计算了上涨与下跌的空间相比，streaks指标则强调了上涨和下跌的时间周期 -- 只有一轮趋势的空间和时间都到位，才有更有可能发生反转；而PercentRank往往可以寻找到行情的加速阶段：一旦出现明显的涨跌异常 -- 这也是一段行情进入尾声的标志之一。正因为如此，Conners RSI才被认为是更有效的摆动类指标，可以比较精准地捕捉顶和底。

Conners RSI的实际表现如何？我们拿最近1000天的沪指进行测试：

![50%](https://images.jieyu.ai/images/2023/08/corners_rsi.png)

backtrader的回测表明，最近4年以来（近似于1000个交易日），沪指仅上涨5.76%，但通过cornner's RSI策略抄底逃顶，我们在指数上竟然获得超过44%的收益。如果是对个股进行操作，收益很可能是数倍。

由于这段时间毕竟沪指是上涨的，如果遇到行情极端不好，访策略表现如何？近两年来恒生指数就宛如下跌的飞刀，假设我们要强接飞刀，表现又是怎样？

![50%](https://www.taindicators.com/blog/2022/connors/2800hk.png)
<caption>图片来源: www.taindicators.com</caption>

结论是，还是不要去接下跌的飞刀。但即使如此，即使是遇到下跌的飞刀，corner's RSI的表现也大大超过指数。

from coursea import *
await init()

class ConnerRSIStrategy(BaseStrategy):
    def __init__(self, url: str, sec: str, **kwargs):
        self.sec = sec
        self.pstreak = kwargs.get("pstreak", 2)
        self.prank = kwargs.get("prank", 20)
        
        self.low_watermark = kwargs.get("low_watermark", 13)
        self.high_watermark = kwargs.get("high_watermark", 58)
        super().__init__(url, **kwargs)
        
    
    async def predict(self, frame: Frame, frame_type: FrameType, i: int, barss, **kwargs):
        bars = barss[self.sec]
        close = bars["close"]
        
        rsi = ta.RSI(close, 6)
        streak = Streak(close)
        rsi_streak = ta.RSI(streak, self.pstreak)
        
        pclose = close[-self.prank:]
        prank = fsum(x < pclose[-1] for x in pclose)/ pclose
        
        crsi = (rsi[-1] + rsi_streak[-1] + prank)/3.0
        
        print("crsi is", crsi)
        if crsi <= self.low_watermark:
            await self.buy(self.sec, money=self.cash, order_time=tf.combine_time(frame, 14, 56))
        elif crsi >= self.high_watermark:
            avail = self.available_shares(sec, frame)
            await self.sell(self.sec, vol = avail, order_time=tf.combine_time(frame, 14, 56))
            
start = datetime.date(2023, 1, 4)
end = datetime.date(2023, 4, 14)

cs = ConnerRSIStrategy(cfg.backtest.url, "002344.XSHE", start=start, end=end, frame_type=FrameType.DAY, baseline="399300.XSHE")
await cs.backtest(portfolio = ["002344.XSHE"], n=30)
await cs.plot_metrics()